In [1]:
from helper_fns import *

In [2]:
import os,sys
from IPython.display import Image

In [3]:
dataset = 'simulated'
threshold_type = 'absolute'

In [4]:
cp_types = ['CV+','Split']
model = 'gpr'
bias = 0
tau = 10
ntrial = 100
sim_data_size = 1000
sigma_eps = 5

In [5]:
filler = '/No_bias/sim_data_size_' + str(sim_data_size) + '/'

### Define variables and load in data

In [6]:
simulated_data = pd.read_csv('0.Datasets/simulated/simulated_data.csv')
X_simulated = simulated_data.iloc[:, 0:2].values
Y_simulated = simulated_data.iloc[:, 2].values
n_simulated = len(Y_simulated)

In [7]:
n_total = len(Y_simulated)
n_train = int(0.7*n_total)
n_test = n_total - n_train

### GPR (Backend) Calcs

In [8]:
np.random.seed(98765)

#Standardize data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(eval('X_' + dataset))
Y_scaled = scaler_y.fit_transform(eval('Y_' + dataset).reshape(-1, 1)).ravel()

y_scale_factor = scaler_y.scale_
y_mean = scaler_y.mean_

x_scale_factor = scaler_X.scale_
x_mean = scaler_X.mean_

In [9]:
X_train, Y_train, X_test, Y_test = generate_data_for_trials(ntrial, n_train, n_total, \
                                                 X_scaled, Y_scaled, bias)

In [10]:
# kernel = RationalQuadratic(alpha = 1.0, length_scale = 1.0)
# gp = GaussianProcessRegressor(kernel = kernel, n_restarts_optimizer = 10)

In [11]:
# for i in tqdm.tqdm(range(ntrial)):
    
#     #Get data for ith trial
#     X_train_trial = X_train[i]
#     Y_train_trial = Y_train[i]
#     X_test_trial = X_test[i]
    
#     #Fit to training data
#     gp.fit(X_train_trial, Y_train_trial)
    
#     # Make predictions on new data points
#     Y_pred_scaled, sigma_scaled = gp.predict(X_test_trial, return_std = True)
    
#     #Rescale predictions
#     Y_pred = Y_pred_scaled * y_scale_factor + y_mean
#     sigma = sigma_scaled * y_scale_factor

#     pd.DataFrame(Y_pred).to_csv(dataset + '/No_bias/sim_data_size_1000/GPR/y_pred_' + str(int(i)) + '.csv', index = False)
#     pd.DataFrame(sigma).to_csv(dataset + '/No_bias/sim_data_size_1000/GPR/sigma_pred_' + str(int(i)) + '.csv', index = False)

In [12]:
# sigma_pred_by_trial = []
# Y_pred_by_trial = []

# for i in range(ntrial):
    
#     sigma_pred_by_trial.append(pd.read_csv(dataset + '/No_bias/sim_data_size_1000/GPR/sigma_pred_' + str(int(i)) + '.csv')['0'].values)
#     Y_pred_by_trial.append(pd.read_csv(dataset + '/No_bias/sim_data_size_1000/GPR/Y_pred_' + str(int(i)) + '.csv')['0'].values)

# pd.DataFrame(sigma_pred_by_trial).to_csv(dataset + '/No_bias/sim_data_size_1000/GPR_' + str(ntrial) + 'Trial_' + 'Sigmas.csv', index = False)
# pd.DataFrame(Y_pred_by_trial).to_csv(dataset + '/No_bias/sim_data_size_1000/GPR_' + str(ntrial) + 'Trial_' + 'Preds.csv', index = False)

In [17]:
def results_by_tau(dataset_to_use, filler, muh_fun_name, ntrial, X1_by_trial, Y1_by_trial, PDs_data, Res_data, \
                   threshold_type, tau_to_use, sigma_eps = None):
    
    if (dataset_to_use == 'simulated'):
        prob_true = generate_true_probs(ntrial, X1_by_trial, Y1_by_trial, PDs_data, \
                                        threshold_type, tau_to_use, sigma_eps)
        pd.DataFrame(prob_true).to_csv(dataset_to_use + filler + 'Threshold_type_' + str(threshold_type) \
                                + '/' + muh_fun_name + '/JAWS_coverage_by_trial/true_prob.csv')
        
    prob_methods = [ 'CV_plus',  'CV', 'split']
    
#     jackknife_plus_probs = []
#     jackknife_probs = []
#     wt_jackknife_plus_probs = []
    
    CV_plus_probs = []
    CV_probs = []
    wt_CV_plus_probs = []
    
    split_probs = []
    wt_split_probs = []
    
    for j in tqdm.tqdm(range(ntrial)):
        PDs_itrial = PDs_data[PDs_data['itrial'] == j]
        Res_itrial = Res_data[Res_data['itrial'] == j]
        
        for i in range(len(prob_methods)):
            eval(prob_methods[i] + '_probs').append(generate_prob_results_by_tau(prob_methods[i], PDs_itrial, Res_itrial, threshold_type, tau_to_use))
    
    for i in range(len(prob_methods)):
        pd.DataFrame(eval(prob_methods[i] + '_probs')).to_csv(dataset_to_use + filler + 'Threshold_type_' + str(threshold_type) + '/' +\
                                          muh_fun_name + '/JAWS_coverage_by_trial/' + str(prob_methods[i]) + '.csv')

    return None


In [18]:
results_by_tau(dataset, filler, model, ntrial, X_test* x_scale_factor + x_mean, \
               Y_test* y_scale_factor + y_mean, PDs_all, Res_all, threshold_type, tau, sigma_eps)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\n220572\OneDrive - Munich Re\Documents - Insure AI\3 - Research and Knowledge\Code\JAWS_experiments_Sukrita\Experiment 1\helper_fns.py:422: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_pred = PDs_data[PDs_data['method'] == 'muh_vals_testpoint'][PDs_data['itrial'] == itrial].iloc[:,5:5+len(y_test)].values[0]
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [09:37<00:00,  5.77s/it]


In [14]:
Res_all = pd.read_csv(dataset + filler + model + '_' + str(ntrial) + 'Trial'  +'_Res.csv')
PDs_all = pd.read_csv(dataset + filler + model + '_' + str(ntrial) + 'Trial'  +'_PDs.csv')
    


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\n220572\OneDrive - Munich Re\Documents - Insure AI\3 - Research and Knowledge\Code\JAWS_experiments_Sukrita\Experiment 1\helper_fns.py:422: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_pred = PDs_data[PDs_data['method'] == 'muh_vals_testpoint'][PDs_data['itrial'] == itrial].iloc[:,5:5+len(y_test)].values[0]
  0%|                                                                                          | 0/100 [00:02<?, ?it/s]


TypeError: cannot convert the series to <class 'float'>

In [ ]:
cv_plus_fn = pd.read_csv(dataset + filler + 'Threshold_type_' + str(threshold_type) + '/' + \
                            model + '/JAWS_coverage_by_trial/CV_plus.csv', index_col=0)

split_fn = pd.read_csv(dataset + filler + 'Threshold_type_' + str(threshold_type) + '/' + \
                            model + '/JAWS_coverage_by_trial/split.csv', index_col=0)

mean_coverage_by_trial = pd.DataFrame([cv_plus_fn.mean(axis = 1).values, \
                                       split_fn.mean(axis = 1).values]).T.rename(columns = \
                {0:'CV+', 1 :'Split'})

true_prob_fn = pd.read_csv(dataset + filler + 'Threshold_type_' + str(threshold_type) + '/' + \
                                model + '/JAWS_coverage_by_trial/true_prob.csv', index_col=0)
            
mean_coverage_by_trial['True Probability'] = true_prob_fn.mean(axis = 1).values 

mean_coverage_by_trial.to_csv(dataset + filler + 'Threshold_type_' + str(threshold_type) + '/' + \
                                model + '/JAWS_coverage_by_trial/mean_coverage.csv',index=False)

### Load in estimates

In [ ]:
PDs_all = pd.read_csv(dataset + filler + model + '_' + str(ntrial) + 'Trial'  +'_PDs.csv')
# PDs_all.iloc[:,5:] = Pds_all.iloc[:,5:]*y_scale_factor[0] + y_mean[0]
# PDs_all.to_csv(dataset + filler + model + '_' + str(ntrial) + 'Trial' +'_PDs.csv', index = False)

In [ ]:
Y_pred_by_trial = pd.read_csv(dataset + '/No_bias/sim_data_size_1000/GPR_' + str(ntrial) + 'Trial_' + 'Preds.csv')
sigma_pred_by_trial = pd.read_csv(dataset + '/No_bias/sim_data_size_1000/GPR_' + str(ntrial) + 'Trial_' + 'Sigmas.csv')
GPR_mean_prob = np.mean(2*norm.cdf(tau/sigma_pred_by_trial)-1, axis = 1)
# GPR_mean_prob = pd.DataFrame(GPR_mean_prob).rename(columns = {0:'value'})
# GPR_mean_prob['model'] = 'GPR'
# GPR_mean_prob['Legend'] = ''

In [ ]:
mean_coverage_by_trial = pd.read_csv(dataset + filler + 'Threshold_type_' + str(threshold_type) + '/' + \
                                model + '/JAWS_coverage_by_trial/mean_coverage.csv')

In [ ]:
Y_pred_by_trial == PDs_all[PDs_all['method'] == 'muh_vals_testpoint'].iloc[:,5:(5 + n_test)].reset_index(drop = True)

In [ ]:
mean_coverage_by_trial['GPR'] = GPR_mean_prob

In [ ]:
sns.boxplot(data = mean_coverage_by_trial)

In [ ]:
for i in range(len(mean_coverage_by_trial.columns)):
        plt.hist(mean_coverage_by_trial.iloc[:,i],  
                 alpha = 0.5, 
                 label = mean_coverage_by_trial.columns[i] )
plt.legend(loc = 'upper left')
plt.savefig(dataset + filler + 'Threshold_type_' + str(threshold_type) + '/' + \
                                model +  '/plots/JAWS_mean_results_' + dataset +'.png')
plt.clf()